In [58]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

#enviorement: nosql3.8

api_key="cfb0e81f9edc5ff91586f4672ba286fb"

# base_url=f"https://api.openweathermap.org/data/2.5/weather?q=Madrid&units=metric&appid={api_key}"

# response=(requests.get(base_url, params={"api_key":api_key})).json()

base_url=f"https://api.openweathermap.org/data/2.5/weather?q=Sudafrica&units=metric"

response1=(requests.get(base_url, params={"appid":api_key})).json()

df=pd.json_normalize(response1)

df.head()

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.humidity,main.sea_level,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1764170584,7200,953987,Republic of South Africa,200,26,-30,...,17,1011,852,3.03,303,4.2,4,ZA,1764126491,1764176329


City name

Temperature (°C)

Humidity (%)

Wind speed (m/s or km/h)

Weather description (e.g., “clear sky”)

In [59]:
requests=df[['name','main.temp', 'main.humidity','wind.speed', 'sys.sunrise']]
requests

,name,main.temp,main.humidity,wind.speed,sys.sunrise
0,Republic of South Africa,28.38,17,3.03,1764126491


In [60]:
paises_por_continente = [
    "Egipto",
    "Nigeria",
    "Sudafrica",
    "Canadá",
    "Brasil",
    "México",
    "Argentina",
    "Japon",
    "India",
    "China",
    "Tailandia",
    "Francia",
    "Alemania",
    "España",
    "Colombia",
    "bielorrusia",
    "Francia",
    "Islandia",
    "Rusia",
    "Peru"
]

resultado=[]

for i in paises_por_continente:

    base_urli=f"https://api.openweathermap.org/data/2.5/weather?q={i}&units=metric"

    response=(requests.get(base_urli, params={"appid":api_key})).json()

    df_fila=pd.json_normalize(response)

    df_fila=df_fila[['name','main.temp', 'main.humidity','wind.speed', 'sys.sunrise']]

    resultado.append(df_fila)

df_final=pd.concat(resultado, ignore_index=True)
df_final



    

TypeError: NDFrame.get() got an unexpected keyword argument 'params'

In [61]:
df_final.to_csv('PaisesTempetaura.csv')
df_final=pd.read_csv("/home/sebas/Weather-Data-Analysis-with-a-public-API/Api/PaisesTempetaura.csv")

4. Data Analysis

Answer questions such as:
Which cities are the hottest and coldest right now?


In [62]:
max=df_final["name"][df_final["main.temp"] == df_final["main.temp"].max()] ## hay que hacer un .item() para coger solo para un elemento
min=df_final["name"][df_final["main.temp"] == df_final["main.temp"].min()]


In [63]:
print(f'El pais mas caliente que es: {max.item()} y el pais mas frio es: {min.item()}')

El pais mas caliente que es: Nigeria y el pais mas frio es: Russia


Is there a relationship between temperature and humidity?

In [64]:
df_final.columns

Index(['Unnamed: 0', 'name', 'main.temp', 'main.humidity', 'wind.speed',
       'sys.sunrise'],
      dtype='object')

In [65]:
## Acordarse si se usa f "" al usar los valores dentro se usa ' '
print(f"La relacion que hay que hay entre temperatura y humedad es {df_final['main.temp'].corr(df_final['main.humidity']).round(4)}")

La relacion que hay que hay entre temperatura y humedad es -0.6582


Which continent seems to have the windiest cities?

In [66]:
paises_africa = [
    "Egipto",
    "Nigeria",
    "Sudafrica"
]

paises_america = [
    "Canadá",
    "México",
    "Colombia",
    "Peru",
    "Brasil",
    "Argentina"
]

paises_asia = [
    "Japon",
    "India",
    "China",
    "Tailandia"
]

paises_europa = [
    "Francia",
    "Alemania",
    "España",
    "Bielorrusia",
    "Islandia"
    ]

total=[paises_africa,paises_america,paises_asia,paises_europa]

resultadoTotal=[]
for paises in total:

    df1= df_final[df_final['name'].isin(paises)]
    max_aire=df1["name"][df1["wind.speed"] == df1["wind.speed"].max()]
    resultadoTotal.append(max_aire.item())

print(f'Los pases con más viento son: {resultadoTotal[0]}, {resultadoTotal[1]}, {resultadoTotal[2]} y {resultadoTotal[3]}')

Los pases con más viento son: Nigeria, Peru, China y Islandia


What’s the most common weather description (e.g., clear, cloudy, rainy)?

In [ ]:
df_final=df_final.drop(columns="Unnamed: 0")


In [74]:
df_final.columns

Index(['name', 'main.temp', 'main.humidity', 'wind.speed', 'sys.sunrise'], dtype='object')

In [ ]:
print(f'El ')